In [28]:
import sys
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
import ee
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import re

def read_file(filename):
    with open(filename) as f:
        contents = f.readlines()

    data = []
    for line in contents:
        if line[0:1] == "#":
            continue
        else:
            data.append(line)

    headers = [x.replace("Soil Moisture Percent","smp").replace(" ","_") for x in data[0].split(",")]
    cols = [x.strip("\n").split(",") for x in data[1:]]

    df = pd.DataFrame(cols, columns = headers)
    
    return df

def get_site_lat_lons(site_num, stations_csv):
    df = pd.read_csv(stations_csv)
    site = df[df['site_name'].str.contains(site_no)]
    
    lat, lon = site['lat'].values, site['lon'].values
    return lat[0], lon[0]


csvs = glob.glob("*.csv")
txts = glob.glob("*.txt")
site_file = txts[0]
stations_csv = csvs[0]

site_no = ''.join(c for c in site_file if c.isdigit())
data = read_file(site_file)
lat, lon = get_site_lat_lons(site_no,stations_csv)

In [3]:
df = pd.read_csv(csvs[0])

In [49]:
names = list(df.site_name)
states = list(df.state)

In [46]:
site_ids = []
for i, x in enumerate(names):
    site_id = re.findall("\d+", names[i])[-1]
    site_ids.append(site_id)

In [61]:
for i, j in enumerate(site_ids[:1]):
    print(site_ids[i], states[i])
    query_string = '''https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport,metric/hourly/start_of_period/{}:{}:SCAN%7Cid=%22%22%7Cname/-35315,-11/SMS:-2:value,SMS:-4:value,SMS:-8:value,SMS:-20:value,SMS:-40:value '''.format(site_ids[i], states[i])
    
    command = '''curl {} --output {}'''.format(query_string, str(site_ids[i]) +".txt")
    
    print(command)
    os.system(command)

2065 AK
curl https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport,metric/hourly/start_of_period/2065:AK:SCAN%7Cid=%22%22%7Cname/-35315,-11/SMS:-2:value,SMS:-4:value,SMS:-8:value,SMS:-20:value,SMS:-40:value  --output 2065.csv
